In [3]:
import numpy as np
import pandas as pd
import swifter
from sklearn.feature_extraction.text import TfidfVectorizer
from process import preproccessing
import pickle
import warnings
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /Users/akest/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('posts.csv')

In [5]:
df

,Unnamed: 0,post_id,title,text,date,views_count,comments_count,bookmarks_count,rating,author_nickname
0,0,365293,LinkedIn автоматизировал добавление в профиль ...,Разработчики LinkedIn объявили о появившейся в...,2015-03-26 16:32:00,7300,1,10,+8,Teachbase
1,1,5005,MobileFaker: фальшивые звонки как социальное о...,Со своего мобильника вы заказываете фальшивый ...,2007-01-26 14:39:00,315,17,2,+16,alizar
2,2,125101,Jelastic is the new kind of Java hosting platform,Jelastic — облачный хостинг для ранее разработ...,2011-07-28 16:40:00,1300,34,17,+29,sirus
3,3,375301,В Mozilla работают над версией Firefox OS для ...,\n\nFirefox OS — относительно новая операционн...,2015-01-09 14:56:00,4800,3,6,+8,marks
4,4,130105,Как я спам слал,Введение\nВ этой небольшой статье я хочу расск...,2011-10-10 18:35:00,25500,13,26,+19,Nike01
...,...,...,...,...,...,...,...,...,...,...
232122,232122,125096,Как научить Wordpress движок дружить с Proxy? ...,Я занимаюсь разработкой локального корпоративн...,2011-07-28 11:12:00,3800,5,6,–1,9k1
232123,232123,125097,Опубликованы обучающие материалы по Visual Stu...,"\nПривет, Хабр! \n\nБуквально вчера мы опублик...",2011-07-28 11:17:00,11200,15,19,+14,Parnassus
232124,232124,125098,PostSharp. Отложенная загрузка зависимостей,"Кусок кода, представленный ниже, вы наверняка ...",2011-07-28 11:47:00,1700,13,17,+4,sidristij
232125,232125,125099,Обзор клавиатур для контроллера PS3,"В принципе, накладные клавиатуры для игровых к...",2011-07-28 13:13:00,1800,18,1,+1,IrinaOcean


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232127 entries, 0 to 232126
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       232127 non-null  int64 
 1   post_id          232127 non-null  int64 
 2   title            232127 non-null  object
 3   text             231467 non-null  object
 4   date             232127 non-null  object
 5   views_count      232127 non-null  int64 
 6   comments_count   232127 non-null  int64 
 7   bookmarks_count  232127 non-null  int64 
 8   rating           232127 non-null  object
 9   author_nickname  232120 non-null  object
dtypes: int64(5), object(5)
memory usage: 17.7+ MB


Дропнем строки с пропущенными значениями 

In [7]:
df.dropna(inplace=True)

Оставим только нужные столбцы

In [8]:
df = df[['title', 'text', 'rating']]

Переведем рейтинг в [0, 1]. Отсортируем по нему

In [9]:
df['rating'] = pd.to_numeric(df['rating'].apply(lambda s: s.replace('–', '-')))
df['rating'] = df['rating'] + np.abs(min(df['rating']))
df['rating'] = df['rating'] / max(df['rating'])

In [10]:
df = df.sort_values('rating', ascending=False, ignore_index=True)

Тут совсем много строк для данных с таким большим объемом текста(статьи). Я даже с параллельной обработкой буду ждать очень долго. Поэтому я сокращу их, чтобы получилось чуть больше 100 мб. Я решил взять 10000 самых рейтинговых статей.

In [11]:
df = df.head(10000)
df.head()

,title,text,rating
0,Делаем приватный монитор из старого LCD монитора,\n\nВы наконец-то можете сделать кое-что со св...,1.000000
1,Были получены исходники 3300 глобальных интерн...,Пару месяцев назад нами (2Товарища и Антон Иса...,0.819742
2,История игрушки. Поле Чудес,"Случилось это в городе, закрытом от шпионов, ц...",0.671980
3,"[Обновлено в 10:52, 14.12.19] В офисе Nginx пр...",Другие материалы по теме:\nEng version\nЧто зн...,0.597180
4,Как Денис Крючков выкупил Хабр у Mail.ru,Денис Крючков (deniskin) рассказал Roem.ru о т...,0.591662


Предобработаем текстовые данные. Хотелось бы ускорить обработку. Для этого решил использовать swifter

In [12]:
df['preproc_text'] = df['text'].swifter.apply(preproccessing)
df['preproc_title'] = df['title'].swifter.apply(preproccessing)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

Векторизуем наши текстовые данные

In [13]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df.preproc_text)

TfidfVectorizer()

In [14]:
vectorizer_title = TfidfVectorizer()
vectorizer_title.fit(df.preproc_title)

TfidfVectorizer()

Запишем их с помощью pickle в файл

In [15]:
f = open('pickled', 'wb')
data = {'titles': df['title'].to_list(),
        'ratings': df['rating'].to_list(), 'text_model': vectorizer, 'texts': df['text'].to_list(),
       'texts_processed': df['preproc_text'].to_list(), 'titles_processed': df['preproc_title'].to_list(),
       'title_model': vectorizer_title}
pickle.dump(data, f)
f.close()